# Simulation for Salt intrusion


First the nessecary python packages are imported. This is a combination of open source packages (numpy, pygimli, ..) and code developed by Deltares (ert_inversion)

In [3]:
import os
import subprocess
from importlib import resources
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pygimli as pg
from pygimli.meshtools import createWorld, readGmsh
from pygimli.physics import ert

from ertoolbox import MPT, inversion

Set the data folder belonging to the ERToolbox package and also set some plotting settings


In [2]:
data_folder = Path(resources.files("ertoolbox.data"))

figsize = (12, 4)
cmin = 0.01
cmax = 3
aspect = 'auto'
cmap = 'viridis'

: 

## Mesh

To be able to simulate for the real geometry of the field measurements, the inversion mesh is created using GMSH and imported using Pygimli. The input file is .geo file and is rewritten into a .msh. After importing the mesh, the prior resistivity values are set in the K value per mesh domain. 

Creating a working mesh is very important, some key aspects that are vital for the inversion to work are:
- mesh should consists of points, lines, line loops and a plane surface, that are also made 'physical'
- line loop should be a closed loop
- electrode positions should be a point defined in the input file
- if you cannot make the mesh in gmsh already, the geometry is corrupt

We use 2 files. 1 file only has 1 fixed conductivity in the inversion domain. The other one has a fresh-salt transition and uses 6 different conductivities. 


The .geo file in imported, translated to .msh and filled with K values in this part of the code:

In [ ]:
filename_1l = data_folder / "kornwerderzand_1l.geo"
meshname_1l = filename_1l.with_suffix(".msh")

geom_1l = subprocess.call(["gmsh", "-format", "msh2", "-2", "-o", meshname_1l, filename_1l], shell=True)
mesh_1l = readGmsh(meshname_1l, verbose=True)
K_1l= [[2,2.7]]

pg.show(mesh_1l, markers=True, clipBoundaryMarkers=True, figsize=(12,4), aspect=aspect)
pg.show(mesh_1l, data = K_1l, clipBoundaryMarkers=True, showMesh=True, figsize=(12,4), aspect=aspect, cMin = cmin, cMax = cmax, cmap = cmap)

: 

: 

In [ ]:
import pygimli.meshtools as mt

filename_6l = data_folder / "kornwerderzand_6l.geo"
meshname_6l = filename_6l.with_suffix(".msh")

geom_6l = subprocess.call(["gmsh", "-format", "msh2", "-2", "-o", meshname_6l, filename_6l], shell=True)
mesh_6l = readGmsh(meshname_6l, verbose=True)

mesh_6l = mt.appendTriangleBoundary(mesh_6l, xbound=100., ybound=100., quality=31, smooth=True,
                                 marker=3, isSubSurface=True, addNodes=5)

K_6l= [[100,2.7],[200,2],[300,1.8],[400,1.5],[500,1],[600,0.6] ]
# K_6l= [[1,100],[3,100]]
pg.show(mesh_6l, markers=True, clipBoundaryMarkers=False, figsize=(12,4), aspect=aspect)
pg.show(mesh_6l, data =K_6l, clipBoundaryMarkers=False, showMesh=True, figsize=(12,4), aspect=aspect, cMin = cmin, cMax = cmax, cMap=cmap)

: 

: 

for the inversion, we use a clean mesh.

In [ ]:
#filename = data_folder / "kornwerderzand_1l_inversion.geo" # lc = 1.0 m
filename = data_folder / "kornwerderzand_1l_inversion.geo" # lc = 0.5 m

inversion_mesh = filename.with_suffix(".msh")

geom = subprocess.call(["gmsh", "-format", "msh2", "-2", "-o", inversion_mesh, filename])
mesh = readGmsh(inversion_mesh, verbose=True)
mesh = mt.appendTriangleBoundary(mesh, xbound=100., ybound=100., quality=31, smooth=True,
                                 marker=3, isSubSurface=True, addNodes=50)

pg.show(mesh, markers=True, clipBoundaryMarkers=True, figsize=(12,4), aspect=aspect)
pg.show(mesh, clipBoundaryMarkers=True, showmesh=True, figsize=(12,4), aspect=aspect)

: 

: 

## Simulate data

Using the custom mesh, we can make simulation data. This consists of the electrode positions and the inversion scheme. 3 schemes are shown here: dipole-dipole (dd), multiple gradient(gr) and one costom made mesh that was used in the field test.

Other possible standard configurations are: 
Wenner Alpha (C-P-P-C) :  wa
Wenner Beta(C-C-P-P) :  wb
Dipole Dipole (CC-PP) :  dd
Schlumberger(C-PP-C) :  slm
Pole Pole (C-P) :  pp
Pole Dipole (C-PP) :  pd
Half Wenner (C-P-P) :  hw
MultipleGradient(C--P-P--C) :  gr


first, the electrode positions are defined:

In [ ]:

points =[(12.99, -1.6041734693877547), (13.867040816326531, -1.6551428571428568), (14.744081632653062, -2.0119285714285713),  (15.621122448979591, -2.0628979591836734), (16.498163265306122, -2.419683673469388), (17.375204081632653, -2.4706530612244895), (18.252244897959184, -2.827438775510204), (19.129285714285714, -2.878408163265306), (20.883367346938776, -3.286163265306122), (21.760408163265307, -3.5410102040816325), (22.637448979591838, -3.5919795918367345), (23.514489795918365, -3.7448877551020407), (24.391530612244896, -3.795857142857143), (25.268571428571427, -3.948765306122449), (26.145612244897958, -3.999734693877551), (27.89969387755102, -4.101673469387755), (28.77673469387755, -4.254581632653061), (29.65377551020408, -4.3055510204081635), (30.530816326530612, -4.356520408163266), (31.407857142857143, -4.407489795918368), (32.284897959183674, -4.45845918367347), (33.16193877551021, -4.509428571428572), (34.038979591836735, -4.662336734693877), (35.7930612244898, -4.764275510204081), (36.67010204081633, -4.815244897959183), (37.54714285714286, -4.9681530612244895), (39.30122448979592, -4.9681530612244895), (40.178265306122455, -5.019122448979592), (41.05530612244898, -5.070091836734694), (42.809387755102044, -5.070091836734694), (43.68642857142858, -5.121061224489796), (44.563469387755106, -5.172030612244898), (46.31755102040817, -5.172030612244898), (48.07163265306123, -5.172030612244898), (49.82571428571429, -5.172030612244898), (50.702755102040825, -5.223), (52.45683673469388, -5.223), (53.333877551020414, -5.172030612244898), (55.08795918367347, -5.172030612244898), (55.965, -5.121061224489796), (56.84204081632653, -5.070091836734694), (58.59612244897959, -5.070091836734694), (59.47316326530613, -5.019122448979592), (60.350204081632654, -4.9681530612244895), (62.104285714285716, -4.9681530612244895), (62.98132653061225, -4.9171836734693875), (63.85836734693878, -4.866214285714285), (65.61244897959183, -4.866214285714285), (66.48948979591836, -4.815244897959183), (67.3665306122449, -4.764275510204081), (69.12061224489796, -4.764275510204081), (69.9976530612245, -4.713306122448979), (70.87469387755102, -4.662336734693877), (72.62877551020408, -4.662336734693877), (73.5058163265306, -4.611367346938775), (74.38285714285715, -4.560397959183673), (76.1369387755102, -4.560397959183673), (77.01397959183673, -4.509428571428572), (77.89102040816326, -4.45845918367347), (79.64510204081633, -4.45845918367347), (80.52214285714285, -4.407489795918368), (81.39918367346938, -4.356520408163266), (83.15326530612245, -4.356520408163266), (84.03030612244898, -4.3055510204081635), (84.9073469387755, -4.254581632653061), (86.66142857142857, -4.254581632653061), (87.5384693877551, -4.203612244897959), (88.41551020408163, -4.152642857142857), (89.29255102040815, -3.897795918367347), (90.1695918367347, -3.846826530612245), (91.04663265306122, -3.4900408163265304),  (91.92367346938775, -3.4390714285714283), (92.80071428571429, -3.0822857142857143), (93.67775510204082, -3.0313163265306122), (94.55479591836735, -2.4706530612244895), (95.43183673469387, -2.419683673469388), (96.3088775510204, -1.859020408163265), (97.18591836734693, -1.808051020408163), (98.06295918367346, -1.1454489795918361), (98.94, -1.0944795918367345)]
len(points)

: 

: 

In [ ]:
#TODO include functions in the ertoolbox, generalized to any distance

# Functie om de afstand tussen twee punten te berekenen
def distance(p1, p2):
    return np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

# Functie om nieuwe punten te genereren die 1 meter uit elkaar liggen
def generate_points(points):
    new_points = [points[0]]

    for i in range(1, len(points)):
        p1 = new_points[-1]
        p2 = points[i]

        while distance(p1, p2) > 1:
            dx = p2[0] - p1[0]
            dz = p2[1] - p1[1]
            dist = distance(p1, p2)

            new_x = p1[0] + dx / dist
            new_z = max(p1[1] + dz / dist, p2[1]) + 0.1

            new_points.append((new_x, new_z))
            p1 = new_points[-1]

    return new_points

def generate_points05(points):
    new_points = [points[0]]

    for i in range(1, len(points)):
        p1 = new_points[-1]
        p2 = points[i]

        while distance(p1, p2) > 0.5:  # Verander 1 naar 0.5
            dx = p2[0] - p1[0]
            dz = p2[1] - p1[1]
            dist = distance(p1, p2)

            new_x = p1[0] + dx / dist * 0.5  # Voeg de 0.5m afstand toe
            new_z = p1[1] + dz / dist * 0.5 +0.1  # Voeg de 0.5m afstand toe

            new_points.append((new_x, new_z))
            p1 = new_points[-1]

    return new_points

: 

: 

In [ ]:
# electrodes 1 m spaced on the bottom of kornwerderzand sluice exit
new_points = generate_points(points)
#electrodes = [[x, y] for x, y in new_points]
# halfplus = int((len(electrodes)/2)+32)
# halfmin = int((len(electrodes)/2)-32)

# electrodes 0.5 m spaced on the bottom of kornwerderzand sluice exit
new_points05 = generate_points05(points)
electrodes = [[x, y] for x, y in new_points05]

halfplus = int((len(electrodes)/2)+64)
halfmin = int((len(electrodes)/2)-64)

electrodes = electrodes[halfmin:halfplus]

plt.plot(new_points, 'ro')
plt.plot(points, 'bo')
plt.plot(electrodes, 'go')
print(electrodes)



# electrodes on the hypothetical water level, for testing purposes of a more standard high conductive 'geological' layer
# xelec = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
#xelec = [24,25,26,24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88]
#zelec = np.zeros(len(xelec))
#electrodes = [[x, y] for x, y in zip(xelec,zelec)]



: 

: 

### Dipole Dipole

The scheme is built-in and data is created using the dipole-dipole scheme for both geometries. Next, the simulated data is inverted using a clean inversion mesh.

In [ ]:
scheme_dd = ert.createData(elecs=electrodes, schemeName='slm')
scheme_gr = ert.createData(elecs=electrodes, schemeName='slm')
simdata_dd_6l = ert.simulate(mesh=mesh_6l, scheme=scheme_dd, res=K_6l, noiseLevel=0.01, noiseAbs=1e-10, seed=1337)
ert.show(simdata_dd_6l)
simdata_dd_6l.remove(simdata_dd_6l['rhoa'] < 0)
inversion_dd_6 = inversion(simdata_dd_6l, mesh, saveresult=False)


: 

: 

In [ ]:
pg.show(mesh, data=inversion_dd_6.model, figsize=figsize, aspect=aspect, cMin=cmin, cMax=cmax, cMap=cmap)

: 

: 

In [ ]:
scheme_dd = ert.createData(elecs=electrodes, schemeName='wa')
simdata_dd_1l = ert.simulate(mesh=mesh_1l, scheme=scheme_dd, res=K_1l, noiseLevel=0.01, noiseAbs=1e-6, seed=1337)
simdata_dd_1l.remove(simdata_dd_1l['rhoa'] < 0)
inversion_dd_1 = inversion(simdata_dd_1l, mesh, saveresult=False)

: 

: 

In [ ]:
pg.show(mesh, data=inversion_dd_1.model, figsize=figsize, aspect=aspect, cMin=1.5, cMax=3.5, cMap='tab10')

: 

: 

### Gradient
for the gradient based scheme, the workflow is the same as for dipole dipole.

In [ ]:
scheme_gr = ert.createData(elecs=electrodes, schemeName = 'gr')

simdata_gr_1l = ert.simulate(mesh=mesh_1l, scheme=scheme_gr, res=K_1l, noiseLevel=1, noiseAbs=1e-6, seed=1337)
simdata_gr_6l = ert.simulate(mesh=mesh_6l, scheme=scheme_gr, res=K_6l, noiseLevel=1, noiseAbs=1e-6, seed=1337)

simdata_gr_1l.remove(simdata_gr_1l['rhoa'] < 0)
simdata_gr_6l.remove(simdata_gr_6l['rhoa'] < 0)

inversion_gr_1 = inversion(simdata_gr_1l, mesh, saveresult=False)
inversion_gr_6 = inversion(simdata_gr_6l, mesh, saveresult=False)


: 

: 

In [ ]:
# fig, ax = plt.subplots(figsize=figsize)
# inversion_gr_1.showResult(ax=ax, aspect=aspect, cMin=cmin, cMax=cmax, cMap=cmap)

pg.show(mesh, data=inversion_gr_1.model, figsize=figsize, aspect=aspect, cMin=cmin, cMax=cmax, cMap=cmap)

#fig, ax = plt.subplots(figsize=figsize)
#inversion_gr_6.showResult(ax=ax, aspect=aspect, cMin=cmin, cMax=cmax, cMap=cmap)

pg.show(mesh, data=inversion_gr_6.model, figsize=figsize, aspect=aspect, cMin=cmin, cMax=cmax, cMap=cmap)

: 

: 

### Customized

For the customized scheme, this scheme should be created first and written down in a datacontainer for Pygimli to be able to handle the scheme.

In [ ]:
#datacontainer for custom abmn

import pygimli as pg
from pygimli.physics import ert


def prepare_custom_scheme(EX,EZ):
    dc = pg.DataContainerERT()

    for s in range(len(EX)):
        dc.createSensor(
            [
                EX[s],
                0,
                EZ[s],
            ]
        )

    A = [2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,32,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,36,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,42,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,43,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,45,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,46,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,53,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,56,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,57,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,58,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,62,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,66,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,68,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,74,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,78,78,78,78,78,78,78,78,78,78,78,78,78,78,79,79,79,79,79,79,79,79,79,79,79,79,79,80,80,80,80,80,80,80,80,80,80,80,80,80,81,81,81,81,81,81,81,81,81,81,81,81,81,82,82,82,82,82,82,82,82,82,82,82,82,83,83,83,83,83,83,83,83,83,83,83,84,84,84,84,84,84,84,84,84,84,85,85,85,85,85,85,85,85,85,86,86,86,86,86,86,86,86,87,87,87,87,87,87,87,88,88,88,88,88,88,88,89,89,89,89,89,89,90,90,90,90,90,91,91,91,91,92,92,92,93,93,94
]
    B = [1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,3,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5,5,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,6,6,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,6,7,7,7,7,7,7,7,5,5,5,5,5,5,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,8,8,6,6,6,6,6,6,7,7,7,7,7,7,8,8,8,8,8,8,9,9,9,9,9,9,9,7,7,7,7,7,7,8,8,8,8,8,8,9,9,9,9,9,9,10,10,10,10,10,10,10,8,8,8,8,8,8,9,9,9,9,9,9,10,10,10,10,10,10,11,11,11,11,11,11,11,9,9,9,9,9,9,10,10,10,10,10,10,11,11,11,11,11,11,12,12,12,12,12,12,12,10,10,10,10,10,10,11,11,11,11,11,11,12,12,12,12,12,12,13,13,13,13,13,13,13,11,11,11,11,11,11,12,12,12,12,12,12,13,13,13,13,13,13,14,14,14,14,14,14,14,12,12,12,12,12,12,13,13,13,13,13,13,14,14,14,14,14,14,15,15,15,15,15,15,15,13,13,13,13,13,13,14,14,14,14,14,14,15,15,15,15,15,15,16,16,16,16,16,16,16,14,14,14,14,14,14,15,15,15,15,15,15,16,16,16,16,16,16,17,17,17,17,17,17,17,15,15,15,15,15,15,16,16,16,16,16,16,17,17,17,17,17,17,18,18,18,18,18,18,18,16,16,16,16,16,16,17,17,17,17,17,17,18,18,18,18,18,18,19,19,19,19,19,19,19,17,17,17,17,17,17,18,18,18,18,18,18,19,19,19,19,19,19,20,20,20,20,20,20,20,18,18,18,18,18,18,19,19,19,19,19,19,20,20,20,20,20,20,21,21,21,21,21,21,21,19,19,19,19,19,19,20,20,20,20,20,20,21,21,21,21,21,21,22,22,22,22,22,22,22,20,20,20,20,20,20,21,21,21,21,21,21,22,22,22,22,22,22,23,23,23,23,23,23,23,21,21,21,21,21,21,22,22,22,22,22,22,23,23,23,23,23,23,24,24,24,24,24,24,24,22,22,22,22,22,22,23,23,23,23,23,23,24,24,24,24,24,24,25,25,25,25,25,25,25,23,23,23,23,23,23,24,24,24,24,24,24,25,25,25,25,25,25,26,26,26,26,26,26,26,24,24,24,24,24,24,25,25,25,25,25,25,26,26,26,26,26,26,27,27,27,27,27,27,27,25,25,25,25,25,25,26,26,26,26,26,26,27,27,27,27,27,27,28,28,28,28,28,28,28,26,26,26,26,26,26,27,27,27,27,27,27,28,28,28,28,28,28,29,29,29,29,29,29,29,27,27,27,27,27,27,28,28,28,28,28,28,29,29,29,29,29,29,30,30,30,30,30,30,30,28,28,28,28,28,28,29,29,29,29,29,29,30,30,30,30,30,30,31,31,31,31,31,31,31,29,29,29,29,29,29,30,30,30,30,30,30,31,31,31,31,31,31,32,32,32,32,32,32,32,30,30,30,30,30,30,31,31,31,31,31,31,32,32,32,32,32,32,33,33,33,33,33,33,33,31,31,31,31,31,31,32,32,32,32,32,32,33,33,33,33,33,33,34,34,34,34,34,34,34,32,32,32,32,32,32,33,33,33,33,33,33,34,34,34,34,34,34,35,35,35,35,35,35,35,33,33,33,33,33,33,34,34,34,34,34,34,35,35,35,35,35,35,36,36,36,36,36,36,36,34,34,34,34,34,34,35,35,35,35,35,35,36,36,36,36,36,36,37,37,37,37,37,37,37,35,35,35,35,35,35,36,36,36,36,36,36,37,37,37,37,37,37,38,38,38,38,38,38,38,36,36,36,36,36,36,37,37,37,37,37,37,38,38,38,38,38,38,39,39,39,39,39,39,39,37,37,37,37,37,37,38,38,38,38,38,38,39,39,39,39,39,39,40,40,40,40,40,40,40,38,38,38,38,38,38,39,39,39,39,39,39,40,40,40,40,40,40,41,41,41,41,41,41,41,39,39,39,39,39,39,40,40,40,40,40,40,41,41,41,41,41,41,42,42,42,42,42,42,42,40,40,40,40,40,40,41,41,41,41,41,41,42,42,42,42,42,42,43,43,43,43,43,43,43,41,41,41,41,41,41,42,42,42,42,42,42,43,43,43,43,43,43,44,44,44,44,44,44,44,42,42,42,42,42,42,43,43,43,43,43,43,44,44,44,44,44,44,45,45,45,45,45,45,45,43,43,43,43,43,43,44,44,44,44,44,44,45,45,45,45,45,45,46,46,46,46,46,46,46,44,44,44,44,44,44,45,45,45,45,45,45,46,46,46,46,46,46,47,47,47,47,47,47,47,45,45,45,45,45,45,46,46,46,46,46,46,47,47,47,47,47,47,48,48,48,48,48,48,48,46,46,46,46,46,46,47,47,47,47,47,47,48,48,48,48,48,48,49,49,49,49,49,49,49,47,47,47,47,47,47,48,48,48,48,48,48,49,49,49,49,49,49,50,50,50,50,50,50,50,48,48,48,48,48,48,49,49,49,49,49,49,50,50,50,50,50,50,51,51,51,51,51,51,51,49,49,49,49,49,49,50,50,50,50,50,50,51,51,51,51,51,51,52,52,52,52,52,52,52,50,50,50,50,50,50,51,51,51,51,51,51,52,52,52,52,52,52,53,53,53,53,53,53,53,51,51,51,51,51,51,52,52,52,52,52,52,53,53,53,53,53,53,54,54,54,54,54,54,54,52,52,52,52,52,52,53,53,53,53,53,53,54,54,54,54,54,54,55,55,55,55,55,55,55,53,53,53,53,53,53,54,54,54,54,54,54,55,55,55,55,55,55,56,56,56,56,56,56,56,54,54,54,54,54,54,55,55,55,55,55,55,56,56,56,56,56,56,57,57,57,57,57,57,57,55,55,55,55,55,55,56,56,56,56,56,56,57,57,57,57,57,57,58,58,58,58,58,58,58,56,56,56,56,56,56,57,57,57,57,57,57,58,58,58,58,58,58,59,59,59,59,59,59,59,57,57,57,57,57,57,58,58,58,58,58,58,59,59,59,59,59,59,60,60,60,60,60,60,60,58,58,58,58,58,58,59,59,59,59,59,59,60,60,60,60,60,60,61,61,61,61,61,61,61,59,59,59,59,59,59,60,60,60,60,60,60,61,61,61,61,61,61,62,62,62,62,62,62,62,60,60,60,60,60,60,61,61,61,61,61,61,62,62,62,62,62,62,63,63,63,63,63,63,63,61,61,61,61,61,61,62,62,62,62,62,62,63,63,63,63,63,63,64,64,64,64,64,64,64,62,62,62,62,62,63,63,63,63,63,63,64,64,64,64,64,64,65,65,65,65,65,65,65,63,63,63,63,64,64,64,64,64,64,65,65,65,65,65,65,66,66,66,66,66,66,66,64,64,64,65,65,65,65,65,65,66,66,66,66,66,66,67,67,67,67,67,67,67,65,65,66,66,66,66,66,66,67,67,67,67,67,67,68,68,68,68,68,68,68,66,67,67,67,67,67,67,68,68,68,68,68,68,69,69,69,69,69,69,69,68,68,68,68,68,68,69,69,69,69,69,69,70,70,70,70,70,70,70,69,69,69,69,69,69,70,70,70,70,70,70,71,71,71,71,71,71,71,70,70,70,70,70,70,71,71,71,71,71,71,72,72,72,72,72,72,72,71,71,71,71,71,72,72,72,72,72,72,73,73,73,73,73,73,73,72,72,72,72,73,73,73,73,73,73,74,74,74,74,74,74,74,73,73,73,74,74,74,74,74,74,75,75,75,75,75,75,75,74,74,75,75,75,75,75,75,76,76,76,76,76,76,76,75,76,76,76,76,76,76,77,77,77,77,77,77,77,77,77,77,77,77,77,78,78,78,78,78,78,78,78,78,78,78,78,78,79,79,79,79,79,79,79,79,79,79,79,79,79,80,80,80,80,80,80,80,80,80,80,80,80,81,81,81,81,81,81,81,81,81,81,81,82,82,82,82,82,82,82,82,82,82,83,83,83,83,83,83,83,83,83,84,84,84,84,84,84,84,84,85,85,85,85,85,85,85,86,86,86,86,86,86,86,87,87,87,87,87,87,87,88,88,88,88,88,88,89,89,89,89,89,90,90,90,90,91,91,91,92,92,93
]
    M = [3,4,5,6,7,8,9,11,12,13,14,15,16,4,5,6,7,8,9,10,19,20,21,22,23,24,12,13,14,15,16,17,5,6,7,8,9,10,11,27,28,29,30,31,32,20,21,22,23,24,25,13,14,15,16,17,18,6,7,8,9,10,11,12,28,29,30,31,32,33,21,22,23,24,25,26,14,15,16,17,18,19,7,8,9,10,11,12,13,29,30,31,32,33,34,22,23,24,25,26,27,15,16,17,18,19,20,8,9,10,11,12,13,14,30,31,32,33,34,35,23,24,25,26,27,28,16,17,18,19,20,21,9,10,11,12,13,14,15,31,32,33,34,35,36,24,25,26,27,28,29,17,18,19,20,21,22,10,11,12,13,14,15,16,32,33,34,35,36,37,25,26,27,28,29,30,18,19,20,21,22,23,11,12,13,14,15,16,17,33,34,35,36,37,38,26,27,28,29,30,31,19,20,21,22,23,24,1,1,1,1,1,1,1,34,35,36,37,38,39,27,28,29,30,31,32,2,2,2,2,2,2,1,1,1,1,1,1,1,35,36,37,38,39,40,2,2,3,3,3,3,2,2,2,2,2,2,1,1,1,1,1,1,2,3,3,3,3,4,4,2,3,3,3,3,3,2,2,2,2,2,2,1,1,1,1,1,2,2,3,3,3,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,1,1,1,1,2,2,2,3,3,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,1,1,1,2,2,2,2,3,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,3,1,1,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,3,3,1,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,3,4,2,2,3,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,3,4,4,2,3,3,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,4,3,3,3,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,2,4,4,4,4,4,5,3,3,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,2,3,4,4,4,4,5,5,3,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,2,3,3,4,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,2,3,3,3,4,4,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,2,2,2,3,3,3,3,4,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,4,2,2,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,4,4,2,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,4,5,3,3,4,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,4,5,5,3,4,4,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,5,4,4,4,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,3,5,5,5,5,5,6,4,4,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,3,4,5,5,5,5,6,6,4,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,3,4,4,5,5,5,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,3,4,4,4,5,5,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,3,3,3,4,4,4,4,5,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,5,3,3,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,5,5,3,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,5,6,4,4,5,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,5,6,6,4,5,5,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,6,5,5,5,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,4,6,6,6,6,6,7,5,5,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,4,5,6,6,6,6,7,7,5,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,4,5,5,6,6,6,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,4,5,5,5,6,6,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,4,4,4,5,5,5,5,6,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,6,4,4,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,6,6,4,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,6,7,5,5,6,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,6,7,7,5,6,6,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,7,6,6,6,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,5,7,7,7,7,7,8,6,6,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,5,6,7,7,7,7,8,8,6,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,5,6,6,7,7,7,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,5,6,6,6,7,7,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,5,5,5,6,6,6,6,7,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,7,5,5,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,7,7,5,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,7,8,6,6,7,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,7,8,8,6,7,7,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,8,7,7,7,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,6,8,8,8,8,8,9,7,7,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,6,7,8,8,8,8,9,9,7,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,6,7,7,8,8,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,6,7,7,7,8,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,6,6,6,7,7,7,7,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,8,6,6,7,7,7,7,7,9,9,9,8,8,8,8,8,8,7,7,7,7,8,8,6,7,7,7,7,7,7,9,9,8,8,8,8,8,8,7,7,7,8,8,8,7,7,7,7,7,7,7,9,8,8,8,8,8,9,7,7,8,8,8,8,7,7,7,7,7,7,7,8,8,8,8,9,9,7,8,8,8,8,8,7,7,7,7,7,7,7,8,8,8,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,7,8,8,9,9,9,9,8,8,8,8,8,8,7,7,7,7,7,7,8,8,9,9,9,9,8,8,8,8,8,8,7,7,7,7,7,8,8,9,9,9,9,8,8,8,8,8,8,7,7,7,7,8,8,8,9,9,9,8,8,8,8,8,8,7,7,7,8,8,8,8,9,9,8,8,8,8,8,9,7,7,8,8,8,8,8,9,8,8,8,8,9,9,7,8,8,8,8,8,8,8,8,8,9,9,9,8,8,8,8,8,8,8,8,8,9,9,9,9,8,8,8,8,8,8,8,8,9,9,9,9,9,8,8,8,8,8,8,8,9,9,9,9,9,8,8,8,8,8,8,8,9,9,9,9,8,8,8,8,8,8,9,9,9,9,8,8,8,8,8,9,9,9,9,8,8,8,8,9,9,9,9,8,8,8,9,9,9,9,8,8,9,9,9,9,9,8,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
]
    N = [4,5,6,7,8,9,10,13,14,15,16,17,18,5,6,7,8,9,10,11,22,23,24,25,26,27,14,15,16,17,18,19,6,7,8,9,10,11,12,30,31,32,33,34,35,23,24,25,26,27,28,15,16,17,18,19,20,7,8,9,10,11,12,13,31,32,33,34,35,36,24,25,26,27,28,29,16,17,18,19,20,21,8,9,10,11,12,13,14,32,33,34,35,36,37,25,26,27,28,29,30,17,18,19,20,21,22,9,10,11,12,13,14,15,33,34,35,36,37,38,26,27,28,29,30,31,18,19,20,21,22,23,10,11,12,13,14,15,16,34,35,36,37,38,39,27,28,29,30,31,32,19,20,21,22,23,24,11,12,13,14,15,16,17,3,3,3,3,3,4,2,2,3,3,3,3,2,2,2,2,2,2,1,1,1,1,1,1,1,3,3,3,3,4,4,2,3,3,3,3,3,2,2,2,2,2,2,2,3,4,5,6,7,8,3,3,3,4,4,4,3,3,3,3,3,3,0,1,2,3,4,5,3,4,5,6,7,8,9,3,3,4,4,4,4,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,5,6,7,8,9,0,1,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,6,7,8,9,0,1,2,8,9,0,1,2,1,2,3,4,5,6,4,5,6,7,8,9,7,8,9,0,1,2,3,9,0,1,2,2,3,4,5,6,7,5,6,7,8,9,0,8,9,0,1,2,3,4,0,1,2,3,4,5,6,7,8,6,7,8,9,0,1,9,0,1,2,3,4,5,1,2,4,5,6,7,8,9,7,8,9,0,1,2,0,1,2,3,4,5,6,2,5,6,7,8,9,0,8,9,0,1,2,3,1,2,3,4,5,6,7,6,7,8,9,0,1,9,0,1,2,3,4,2,3,4,5,6,7,8,7,8,9,0,1,2,0,1,2,3,4,5,3,4,5,6,7,8,9,8,9,0,1,2,3,1,2,3,4,5,6,4,5,6,7,8,9,0,9,0,1,2,3,2,3,4,5,6,7,5,6,7,8,9,0,1,0,1,2,3,3,4,5,6,7,8,6,7,8,9,0,1,2,1,2,3,4,5,6,7,8,9,7,8,9,0,1,2,3,2,3,5,6,7,8,9,0,8,9,0,1,2,3,4,3,6,7,8,9,0,1,9,0,1,2,3,4,5,7,8,9,0,1,2,0,1,2,3,4,5,6,8,9,0,1,2,3,1,2,3,4,5,6,7,9,0,1,2,3,4,2,3,4,5,6,7,8,0,1,2,3,4,3,4,5,6,7,8,9,1,2,3,4,4,5,6,7,8,9,0,2,3,4,5,6,7,8,9,0,1,3,4,6,7,8,9,0,1,2,4,7,8,9,0,1,2,3,8,9,0,1,2,3,4,9,0,1,2,3,4,5,0,1,2,3,4,5,1,2,3,4,5,2,3,4,5,3,4,5,4,5,5
]

    for i in range(len(A)):
        dc.addFourPointData(
            A[i]-1,
            B[i]-1,
            M[i]-1,
            N[i]-1,
        )
    k = ert.createGeometricFactors(dc)
    dc.set("k", k)

    return dc

dc = prepare_custom_scheme(EX,EZ)


: 

: 

then, a similar approach of making the data and inverting the data is carried out.

In [ ]:
simdata_c_1l = ert.simulate(mesh=mesh_1l, scheme=dc, res=K_1l, noiseLevel=1, noiseAbs=1e-6, seed=1337)
simdata_c_6l = ert.simulate(mesh=mesh_6l, scheme=dc, res=K_6l, noiseLevel=1, noiseAbs=1e-6, seed=1337)

simdata_c_1l.remove(simdata_c_1l['rhoa'] < 0)
simdata_c_6l.remove(simdata_c_6l['rhoa'] < 0)

inversion_c_1 = inversion(simdata_c_1l, mesh, saveresult=False)
inversion_c_6 = inversion(simdata_c_6l, mesh, saveresult=False)

: 

: 